# Map to analyse Global Land Cover!

In [1]:
import os
import json

import ee
import folium
import plotly.offline as pyo
from shapely.geometry import Polygon, box

from streamlit_map.geocoder import Geocoder
from streamlit_map.geodescriber import GeoDescriber
from streamlit_map.visualize import foliumMapGEE, ipyleafletMapGEE, create_stacked_bar
from streamlit_map.processing import ZonalStatistics
from streamlit_map.data import GEEData

In [2]:
def _get_bbox(geojson: dict) -> box:
    # Create a Shapely polygon from the coordinates
    poly = Polygon(geojson['geometry']['coordinates'][0])
    # Get the bbox coordinates using the bounds() method
    min_x, min_y, max_x, max_y = poly.bounds
    # Create the box object using the box() function
    shapely_box = box(min_x, min_y, max_x, max_y)

    return shapely_box

In [3]:
private_key = json.loads(os.getenv("EE_PRIVATE_KEY"))
ee_credentials  = ee.ServiceAccountCredentials(email=private_key['client_email'], key_data=os.getenv("EE_PRIVATE_KEY"))

In [4]:
ee.Initialize(credentials=ee_credentials)

In [5]:
MAX_ALLOWED_AREA_SIZE = 25.0

In [6]:
datasets = {}
for dataset in ['Global-Land-Cover', 'Koppen-Geiger-Climate']:
    datasets[dataset] = GEEData(dataset)
    
gee_data = datasets['Global-Land-Cover']

In [7]:
m = foliumMapGEE()

for dataset, data in datasets.items():
    m.add_gee_layer(
        image=data.ee_image(),
        sld_interval=data.sld_interval(),
        name=dataset
    )
           
m.add_layer_control()

m

In [11]:
m = ipyleafletMapGEE()

for dataset, data in datasets.items():
    m.add_gee_layer(
        image=data.ee_image(),
        sld_interval=data.sld_interval(),
        name=dataset
    )
m

Draw a rectangle on map to select and area.


ipyleafletMapGEE(center=[25.0, 55.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [9]:
geojson = m.geometry['features']
top = {}
figs = {}
for dataset, data in datasets.items():
    zs = ZonalStatistics(data, MAX_ALLOWED_AREA_SIZE)

    # Call the zs.check_area_and_compute function to get the plotly figure
    stats = zs.check_area_and_compute(geojson=geojson)

    # sort the items from top to bottom and take the top 8 elements
    top_8 = sorted(stats.items(), key=lambda x: x[1], reverse=True)[:8]
    top_8 = {k: v for k, v in top_8}

    top[dataset] = top_8
    
    # Plot figure
    colors = {data.class_names()[key]: item for key, item in
              data.class_colors().items()}

    fig = create_stacked_bar(values=stats, colors=colors)
    
    figs[dataset] = fig

In [9]:
# Display the plotly figure object in the notebook
# Set notebook mode to work in offline
pyo.init_notebook_mode()
for dataset, fig in figs.items():
    pyo.iplot(fig)

In [8]:
# Geocoder
bbox = _get_bbox(geojson=geojson)

# create Geocoder object
geolocator = Geocoder(user_agent="my-app")

# reverse geocode center point of box to get region and country
center_point = bbox.centroid
region, country = geolocator.reverse_geocode(center_point)

print("Region: ", region)
print("Country: ", country)

Region:  Sidama
Country:  Ethiopia


In [9]:
# Geodescribe the region with OpenAI API
geo_describer = GeoDescriber(model_name="text-davinci-003")
description = geo_describer.generate_description(
    land_cover_per=top['Global-Land-Cover'],
    climate_per=top['Koppen-Geiger-Climate'],
    region_name=region,
    country=country
)

In [10]:
from IPython.display import Markdown, display


display(Markdown(f"""
## Description of the region: 

{description}
"""))


## Description of the region: 

The climate in Sidama is most characterized by temperate oceanic and subtropical highland climates which have mild temperatures, high humidity and moderate rainfall throughout the year. The land cover is mostly characterized by cropland, rainfed with significant tree cover in both deciduous and evergreen varieties. Shrubland is also found in the area. The landscape is mainly agricultural with some natural vegetation and water bodies.

The socioeconomic situation in the region is likely quite dependent on the agricultural industry. The population likely depends on subsistence farming and the production of staple crops like cereals, grains and pulses to support the local economy. The region is likely to be impoverished as agricultural yields are heavily impacted by weather patterns, not to mention lack of infrastructure and access to markets for trading produce.
